In [ ]:
import matplotlib
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import os
import umap
import datashader as ds
import colorcet as cc
import igraph
import tqdm
from scipy import sparse
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans

from matplotlib.collections import PolyCollection
from matplotlib.colors import ListedColormap

from dredFISH.Analysis import TissueGraph
from dredFISH.Visualization import Viz
from dredFISH.Utils.__init__plots import * 
from dredFISH.Utils import powerplots
from dredFISH.Utils import miscu
from dredFISH.Utils import tmgu

import importlib
importlib.reload(Viz)
importlib.reload(TissueGraph)

#### Load data

In [ ]:
respath = '/bigstore/GeneralStorage/fangming/projects/dredfish/figures/'

In [ ]:
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1'
!ls -alhtr $basepth
!head $basepth"/TMG.json"

In [ ]:
df = pd.read_csv(
    os.path.join(basepth, "analysis_dev.csv"))
df

In [ ]:
TMG = TissueGraph.TissueMultiGraph(basepath=basepth, 
                                   redo=False, # load existing 
                                  )
TMG

In [ ]:
# spatial coordinates
layer = TMG.Layers[0]
XY = layer.XY
x, y = XY[:,0], XY[:,1]
###
x, y = y, x # a temporary hack
###

cells = layer.adata.obs.index.values

N = layer.N
print(N)
# measured basis
ftrs_mat = layer.feature_mat

# umap_mat = umap.UMAP(n_neighbors=30, min_dist=0.1).fit_transform(ftrs_mat)

# types

# regions

In [ ]:
labels = df['type_r0.1'].values
edges = np.asarray(layer.SG.get_edgelist()) 
ctg, ctg_idx = np.unique(labels, return_inverse=True) 
print(ctg)

i = edges[:,0] # cells
j = ctg_idx[edges[:,1]] # types it connects
dat = np.repeat(1, len(i))

env_mat = sparse.coo_matrix((dat, (i,j)), shape=(N, len(ctg))).toarray() # dense
env_mat = env_mat/env_mat.sum(axis=1).reshape(-1,1)
env_mat = np.nan_to_num(env_mat, 0)
env_mat

In [ ]:
%%time
k_kms = [5, 10] #[2,5,10,20,50,100]
for k_km in tqdm.tqdm(k_kms):
    kmeans = KMeans(n_clusters=k_km, random_state=1)
    reg_clsts = kmeans.fit_predict(env_mat)
    df[f'type_reg_k{k_km}'] = np.char.add('t', np.array(reg_clsts).astype(str))

In [ ]:
for k_km in k_kms:
    hue = f'type_reg_k{k_km}'
    hue_order = np.sort(np.unique(df[hue]))
    ntypes = len(hue_order)

    fig, axs = plt.subplots(1, 2, figsize=(8*2,6))
    fig.suptitle(f"{hue}; n={ntypes}")
    ax = axs[0]
    sns.scatterplot(data=df, x='x', y='y', 
                    hue=hue, hue_order=hue_order, 
                    s=0.5, edgecolor=None, 
                    legend=False,
                    ax=ax)
    # ax.legend(loc='upper left', bbox_to_anchor=(0, -0.1), ncol=5)
    ax.set_aspect('equal')
    ax.axis('off')

    ax = axs[1]
    sns.scatterplot(data=df, x='umap_x', y='umap_y', 
                    hue=hue, hue_order=hue_order, 
                    s=0.5, edgecolor=None, 
                    legend=False,
                    ax=ax)
    # ax.legend(loc='upper left', bbox_to_anchor=(0, -0.1), ncol=5)
    ax.set_aspect('equal')
    ax.axis('off')
    fig.subplots_adjust(wspace=0)
    plt.show()

# Get from region types to regions; and visualize region boundaries 

In [ ]:
k_km = 10 
region_types = df[f'type_reg_k{k_km}'].values
region_types

In [ ]:
polys = TMG.Geoms[0]['poly'] #) #) #[0]
bdbox = np.array(TMG.Geoms[0]['BoundingBox'].exterior.xy).T

# a hack
polys = [np.vstack([poly[:,1], poly[:,0]]).T for poly in polys]
bdbox = np.array(np.vstack([bdbox[:,1], bdbox[:,0]])).T
# end of the hack

mx = np.max(bdbox,axis=0)
mn = np.min(bdbox,axis=0)

ctg, idx = np.unique(region_types, return_inverse=True)
colors = sns.color_palette("Set2", len(ctg))
colors

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
p = PolyCollection(polys, edgecolors=('none'), cmap=ListedColormap(colors)) # cmap=self.clrmp)
p.set_array(idx)

ax.add_collection(p)
ax.set_aspect('equal') #, 'box')
ax.set_xlim([mn[0],mx[0]])
ax.set_ylim([mn[1],mx[1]])
ax.axis('off')
ax.set_title(f"n={len(colors)} region types")

plt.show()

# get regions (region type zones) 

In [ ]:
def get_cell_zones(SG, cell_types):
    """
    """
    N = len(cell_types)
    
    # trim edges -- remove connects from diffent types
    edges = np.asarray(SG.get_edgelist())
    edges_bytype = cell_types[edges]
    edges_sametype = edges[edges_bytype[:,0]==edges_bytype[:,1]]

    # get components (same type and spatially connected); each component is assigned an index
    zones = igraph.Graph(n=N, edges=edges_sametype, directed=False)
    zone_types = np.asarray(zones.components().membership)

    return zone_types

In [ ]:
# region_types.
np.unique(region_types)

In [ ]:
SG = layer.SG
# cell -> type (clustering)
cell_types = region_types
# cell -> zone (continuity)
cell_zones = get_cell_zones(layer.SG, cell_types)
# zone -> types (reindexing)
unqs, idx, invs, cnts, = np.unique(cell_zones, return_index=True, return_inverse=True, return_counts=True)
print(unqs.shape, idx.shape, invs.shape, cnts.shape)
print(np.all(unqs[invs] == cell_zones)) # use invs to recover the original
zone_types = cell_types[idx]

# sanity check
print(np.all(zone_types[cell_zones] == cell_types))
print(cell_types.shape, cell_zones.shape, zone_types.shape)

In [ ]:
# unqs.shape, invs.shape, cnts.shape, 
th = 2
cond = cnts[invs] > th

fig, ax = plt.subplots()
ax.plot(np.log10(np.sort(cnts)))
ax.axhline(np.log10(th), linestyle='--', color='k')
ax.text(0, np.log10(th*1.3), f"n={th}")
ax.set_xlabel('zones')
ax.set_ylabel('log10(# cells in zone)')

In [ ]:
# k_km = 10
# region_types = df[f'type_reg_k{k_km}'].values
region_types = cond

In [ ]:
polys = TMG.Geoms[0]['poly'] #) #) #[0]
bdbox = np.array(TMG.Geoms[0]['BoundingBox'].exterior.xy).T

# a hack
polys = [np.vstack([poly[:,1], poly[:,0]]).T for poly in polys]
bdbox = np.array(np.vstack([bdbox[:,1], bdbox[:,0]])).T
# end of the hack

mx = np.max(bdbox,axis=0)
mn = np.min(bdbox,axis=0)

ctg, idx = np.unique(region_types, return_inverse=True)
colors = sns.color_palette("tab10", len(ctg))
colors

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
p = PolyCollection(polys, edgecolors=('none'), cmap=ListedColormap(colors)) # cmap=self.clrmp)
p.set_array(idx)

ax.add_collection(p)
ax.set_aspect('equal') #, 'box')
ax.set_xlim([mn[0],mx[0]])
ax.set_ylim([mn[1],mx[1]])
ax.axis('off')

ax.set_title(f'region zones with > {th} cells')
plt.show()

In [ ]:
# zone graph 
ZSG = SG.copy()
# comb = {'X': 'mean', 'Y': 'mean'}
ZSG.contract_vertices(cell_zones) # , combine_attrs=comb)
ZSG.simplify()

In [ ]:
zsg_edges = np.asarray(ZSG.get_edgelist())

In [ ]:
zsg_edges_bytype = cnts[zsg_edges] <= th
zsg_edges_difftype = zsg_edges[np.logical_xor(zsg_edges_bytype[:,0], zsg_edges_bytype[:,1])]

# sanity check
print(np.all(np.sum(cnts[zsg_edges_difftype] <= th, axis=1) == 1))

# edges with the second node bad are swapped to the first
e1, e2 = zsg_edges_difftype[:,0].copy(), zsg_edges_difftype[:,1].copy()
cond = cnts[e2]<=th # e2 is bad
e1sub, e2sub = e1[cond], e2[cond]
idxsub = np.arange(len(e1))[cond]
np.put(e1, idxsub, e2sub)
np.put(e2, idxsub, e1sub)
# sanity check
print(np.all(cnts[e1]<=th), np.sum(cnts[e2]<=th))

# uniq
e1u, e1ui = np.unique(e1, return_index=True)
e2u = e2[e1ui]

zone_remap = pd.Series(e2u, index=e1u)
print(zone_remap.shape)


In [ ]:
zones_u = zone_remap.reindex(unqs).fillna(pd.Series(unqs)).astype(int)
zones_u

In [ ]:
zone_types_u = zone_types[zones_u.values]
cell_types_u = zone_types_u[cell_zones]


In [ ]:
# k_km = 10 
region_types = cell_types_u# df[f'type_reg_k{k_km}'].values
region_types

In [ ]:
polys = TMG.Geoms[0]['poly'] #) #) #[0]
bdbox = np.array(TMG.Geoms[0]['BoundingBox'].exterior.xy).T

# a hack
polys = [np.vstack([poly[:,1], poly[:,0]]).T for poly in polys]
bdbox = np.array(np.vstack([bdbox[:,1], bdbox[:,0]])).T
# end of the hack

mx = np.max(bdbox,axis=0)
mn = np.min(bdbox,axis=0)

ctg, idx = np.unique(region_types, return_inverse=True)
colors = sns.color_palette("Set2", len(ctg))
colors

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
p = PolyCollection(polys, edgecolors=('none'), cmap=ListedColormap(colors)) # cmap=self.clrmp)
p.set_array(idx)

ax.add_collection(p)
ax.set_aspect('equal') #, 'box')
ax.set_xlim([mn[0],mx[0]])
ax.set_ylim([mn[1],mx[1]])
ax.axis('off')
ax.set_title(f"n={len(colors)} region types")

plt.show()

In [ ]:
SG = layer.SG
# cell -> type (clustering)
cell_types = region_types
# cell -> zone (continuity)
cell_zones = get_cell_zones(layer.SG, cell_types)
# zone -> types (reindexing)
unqs, idx, invs, cnts, = np.unique(cell_zones, return_index=True, return_inverse=True, return_counts=True)
print(unqs.shape, idx.shape, invs.shape, cnts.shape)
print(np.all(unqs[invs] == cell_zones)) # use invs to recover the original
zone_types = cell_types[idx]

# sanity check
print(np.all(zone_types[cell_zones] == cell_types))
print(cell_types.shape, cell_zones.shape, zone_types.shape)

In [ ]:
# unqs.shape, invs.shape, cnts.shape, 
th = 2
cond = cnts[invs] > th

fig, ax = plt.subplots()
ax.plot(np.log10(np.sort(cnts)))
ax.axhline(np.log10(th), linestyle='--', color='k')
ax.text(0, np.log10(th*1.3), f"n={th}")
ax.set_xlabel('zones')
ax.set_ylabel('log10(# cells in zone)')